In [1]:
# Load Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Geo Map
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import plotly.express as px

df_salesv2 = pd.read_csv('csv/fact_sales_v2.csv', sep=';', encoding='utf-8') # digunakan pada: geomap model asosiasi

In [7]:
df_geocode = df_salesv2.copy(deep=True) # ☠️ Load 32 detik
display(df_geocode.head())

df_geocode['city'] = df_geocode['city'].str.replace(r'^(Kota|Kabupaten)\s+', '', case=False, regex=True).str.strip()
df_geocode = df_geocode[df_geocode['city'].notnull() & (df_geocode['city'] != '')]
display(df_geocode.head())

unique_orders = df_geocode[['order_id', 'city']].drop_duplicates()
city_transaction_counts = unique_orders['city'].value_counts().reset_index()
city_transaction_counts.columns = ['city', 'transaction_count']
display(city_transaction_counts.head())

most_bought_items = (
    df_geocode.groupby(['city', 'order_item_name'])
    .size()
    .reset_index(name='count')
    .sort_values(['city', 'count'], ascending=[True, False])
    .drop_duplicates('city')
    .rename(columns={'order_item_name': 'most_bought_product'})
)

display(most_bought_items.head())

,order_id,order_date,product_id,order_item_name,total_sales,order_item_type,product_qty,product_net_revenue,customer_id,first_name,last_name,email,city,state
0,1181,2025-05-10 08:17:14,1010,Hosofshopaholic - Darel Skirt,7,line_item,1,160000,110,Claresta,Alway,calway1z@wix.com,Tapanuli Tengah,SU
1,1180,2025-05-10 08:13:52,986,ECINOS - Philia Oversized Linen Shirt,2,line_item,1,269000,109,Blythe,Aspinal,baspinal1y@fc2.com,Indragiri Hilir,RI
2,1179,2025-05-10 08:10:36,883,Voal Motif Printing Sublim Segiempat Premium,3,line_item,1,31000,108,Alix,Kirkup,akirkup1x@parallels.com,Makassar,SS
3,1178,2025-05-10 08:08:08,856,URBAN&CO BASIC Sandal Wanita Sendal Teplek Cas...,3,line_item,1,151900,107,Morgana,Randal,mrandal1w@latimes.com,Palembang,SS
4,1177,2025-05-10 08:01:50,860,SHOEBALI Heels 021-218,4,line_item,1,215000,106,Harcourt,Clayson,hclayson1v@yahoo.com,Denpasar,BA


,order_id,order_date,product_id,order_item_name,total_sales,order_item_type,product_qty,product_net_revenue,customer_id,first_name,last_name,email,city,state
0,1181,2025-05-10 08:17:14,1010,Hosofshopaholic - Darel Skirt,7,line_item,1,160000,110,Claresta,Alway,calway1z@wix.com,Tapanuli Tengah,SU
1,1180,2025-05-10 08:13:52,986,ECINOS - Philia Oversized Linen Shirt,2,line_item,1,269000,109,Blythe,Aspinal,baspinal1y@fc2.com,Indragiri Hilir,RI
2,1179,2025-05-10 08:10:36,883,Voal Motif Printing Sublim Segiempat Premium,3,line_item,1,31000,108,Alix,Kirkup,akirkup1x@parallels.com,Makassar,SS
3,1178,2025-05-10 08:08:08,856,URBAN&CO BASIC Sandal Wanita Sendal Teplek Cas...,3,line_item,1,151900,107,Morgana,Randal,mrandal1w@latimes.com,Palembang,SS
4,1177,2025-05-10 08:01:50,860,SHOEBALI Heels 021-218,4,line_item,1,215000,106,Harcourt,Clayson,hclayson1v@yahoo.com,Denpasar,BA


,city,transaction_count
0,Pontianak,58
1,Medan,5
2,Yogyakarta,5
3,Bandung,5
4,Jakarta,4


,city,most_bought_product,count
0,Bandung,Breakside Polo Shirt Gavriel - Darkgrey Polo S...,1
8,Banjarmasin,Baju Kaos Roblox anak Laki-laki cowok,1
15,Bekasi,Compass Velocity Black,1
17,Denpasar,Jam Tangan Pria Kulit,1
19,Depok,Atasan Knit Wanita,1


In [8]:
city_sales = pd.merge(city_transaction_counts, most_bought_items[['city', 'most_bought_product']], on='city', how='left')
city_sales.head()

,city,transaction_count,most_bought_product
0,Pontianak,58,AMOENE [MADE] Long Strap Dress Navy
1,Medan,5,Jam Tangan Wanita JT 8151
2,Yogyakarta,5,Arabian Voile (Hijab Voal) | MINIMSLM
3,Bandung,5,Breakside Polo Shirt Gavriel - Darkgrey Polo S...
4,Jakarta,4,Baju Kaos Roblox anak Laki-laki cowok


In [10]:
geolocator = Nominatim(user_agent="myApp", timeout=10)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def safe_geocode(city):
    try:
        return geocode(f"{city}, Indonesia")
    except:
        return None

city_sales['location'] = city_sales['city'].apply(safe_geocode)
city_sales['latitude'] = city_sales['location'].apply(lambda loc: loc.latitude if loc else None)
city_sales['longitude'] = city_sales['location'].apply(lambda loc: loc.longitude if loc else None)

# Hapus baris tanpa koordinat
city_sales = city_sales.dropna(subset=['latitude', 'longitude'])
city_sales.head()

,city,transaction_count,most_bought_product,location,latitude,longitude
0,Pontianak,58,AMOENE [MADE] Long Strap Dress Navy,"(Pontianak, Kalimantan Barat, Kalimantan, Indo...",-0.022690,109.344749
1,Medan,5,Jam Tangan Wanita JT 8151,"(Kota Medan ᯔᯩᯑᯉ᯲, Sumatera Utara, Sumatera, I...",3.589665,98.673826
2,Yogyakarta,5,Arabian Voile (Hijab Voal) | MINIMSLM,"(Kota Yogyakarta, Daerah Istimewa Yogyakarta, ...",-7.801265,110.364686
3,Bandung,5,Breakside Polo Shirt Gavriel - Darkgrey Polo S...,"(Kota Bandung, Jawa Barat, Jawa, Indonesia, (-...",-6.921553,107.611021
4,Jakarta,4,Baju Kaos Roblox anak Laki-laki cowok,"(Daerah Khusus ibukota Jakarta, Jawa, Indonesi...",-6.175405,106.827168


In [13]:
# city_sales.to_csv('city_sales.csv', index=False)
# df = pd.read_csv('city_sales.csv')
# df

In [3]:
# === PLOT ===
fig = px.scatter_geo(
    city_sales,
    lat='latitude',
    lon='longitude',
    size='transaction_count',
    color_discrete_sequence=['red'],  # Titik warna merah
    hover_name='city',
    hover_data={
        'transaction_count': True,
        'most_bought_product': True,
        'latitude': False,
        'longitude': False
    },
    title='Sebaran Transaksi per Kota',
    projection='mercator'
)

fig.update_geos(
    visible=True,
    lataxis_range=[-11, 6],
    lonaxis_range=[95, 141],
    showcountries=True,
    countrycolor="Black"
)

fig.update_layout(
    margin={"r": 0, "t": 30, "l": 0, "b": 0},
    legend_title_text='Jumlah Transaksi'
)

fig.show()